# How do I _copy_ an app from one of my other _projects_?
### Overview
Apps can be copied from one of two sources:
1. other projects you are a member of
2. public reference. 

Here we focus on copying an app from _another project_. 

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within your project (we will just use that call directly and pick one here).
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy our own App
We will first list all our projects, then list the apps within the first project, and finally copy an app between the first and second project. (Note that here there is a chance to copy also a specific version of the app, by changing the last character of the following variable to the version you want):
```python
    source_app.id
```
The critical information for this POST is the **app_id**. Note, you are **not** allowed to copy the same app **and** assign the same name<sup>1</sup> more than once. If you change the name, it is ok. Here we are using the App's **ID**, alternatively you can match bases on the App's **name** as shown here [apps_copyFromPublicApps](apps_copyFromPublicApps.ipynb)

To make these results very obvious, use an empty project as your my\_project (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or add the _name_ arguement to something ridiculous like 'Orange Mocha Frapachino Maker'<sup>2</sup>

### Warning on the NAME argument
When copying apps, it is also possible to specify the **name** using

```python
    my_new_app = my_app_source.copy(project = my_project, name = my_app_source.name)
```

However, we caution that this can lead to conflicts with similarly named apps. Unless you are _certain_ that you want to set a specific, **different** app name, _please_ omit the name argument. It will still inherit the name from the app.

```python
    my_new_app = my_app_source.copy(project = my_project)
```


<sup>1</sup> Unlike files, apps are accesssible **both** *within* a project (here the Public Reference project) **and** by a *visibility* property (which may be set to 'public')

<sup>2</sup> Note that setting the **name** of an app, actually changes the **id**. This will be fixed sometime in the future.

In [ ]:
# [USER INPUT] Set project names and app_id here:
source_project_name = 'Keep on Smiling'
my_project_name = 'cicipici'
app_id = 'jack_digi/keep-on-smiling/whole-exome-sequencing-gatk-2-3-9-lite/0'


# Identify source and target (my_project) projects
source_project = [p for p in api.projects.query(limit=100).all()
              if p.name == source_project_name]
my_project = [p for p in api.projects.query(limit=100).all()
              if p.name == my_project_name]


# Double-check that both projects exist
if not source_project:
    print('Source project (%s) not found, check spelling' % source_project_name)
    raise KeyboardInterrupt
else:
    source_project = source_project[0]
    
if not my_project:
    print('Target project (%s) not found, check spelling' % my_project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
my_apps = api.apps.query(project = my_project.id, limit=100)
source_app = [a for a in api.apps.query(
        project = source_project.id, limit=100
    ).all() if a.id == app_id][0]

duplicate_app = [a for a in my_apps.all() if a.name == source_app.name]

if duplicate_app:
    print('App already exists in second project, please try another app')
else:
    print('App (%s) does not exist in Project (%s); copying now' % \
          (source_app.name, my_project.name))
    
    new_app = source_app.copy(
        project = my_project.id
    )
        
    # re-list apps in target project to verify the copy worked
    my_app_names = [
        a.name for a in api.apps.query(
            project = my_project.id, limit=100
        ).all()
    ]
    
    if source_app.name in my_app_names:
        print('Sucessfully copied one app!')
    else:
        print('Something went wrong...')  

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-an-app)